# Microsoft Azure Automated ML デモ

## 目的と課題

このノートブックの目的は、Azure Machine Learning（AML）サービス（プレビュー）を使用して、ユーザーが Machine Learning（ML）アプリケーションを構築および展開することです。ここで取り組む課題は、単純な予測保守ソリューションです。

このノートブックには、モデルをロード、準備、トレーニング、および展開するための完全なコードがあります。このデモでは、わずか数分でプロセス全体を実行できるように、小さなパブリックデータセットを選択しました。AML サービスは、あらゆる問題に対処し、新しい機能を追加するために絶えず更新されています。

このノートブックは、すべて Azure でデモを実行できるように、Azure Notebooks サービスを使用してテストされています。

以下は、手順の概要です。

1. データの取得と準備
2. Automated ML
3. モデルの配置

## 1. データの取得と準備
このノートブックでは、NASA 予報センターの Turbo-Fan Failure データセットを使用します。データセットは既にこのプロジェクト内にありますが、元のデータセットは以下で参照できます：
https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/#turbofan

このスクリプトファイルと同じフォルダに .txt ファイルがあります。それを Pandas DataFrame に読み込みます。ヘッダーはスペース区切りのtxtファイルには含まれていないので、zip ファイルの ReadMe から割り当てます。Pamdas では、read_csv メソッドで、スペースで区切られたファイルを delimiter オプションとともに使用します。

In [1]:
import pandas as pd
train = pd.read_csv("train_FD001.txt", delimiter="\s|\s\s", index_col=False, engine='python', names=['unit','cycle','os1','os2','os3','sm1','sm2','sm3','sm4','sm5','sm6','sm7','sm8','sm9','sm10','sm11','sm12','sm13','sm14','sm15','sm16','sm17','sm18','sm19','sm20','sm21'])

データを簡単に見てみます。

In [2]:
train.head(5)

,unit,cycle,os1,os2,os3,sm1,sm2,sm3,sm4,sm5,...,sm12,sm13,sm14,sm15,sm16,sm17,sm18,sm19,sm20,sm21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


データセットにはたくさんのエンジンユニットがあり、各エンジンについて、サイクル（回数）として毎回の飛行がリスト化されています。サイクルはエンジンが故障するまでカウントアップされます。私たちが予測したいのは「故障するまでのサイクル数」です。そのため、RUL、つまり残存有効寿命という新しい列を計算する必要があります。これは、最後のサイクル値から単位あたりの各サイクル値を引いたものになります。

In [3]:
# Assign ground truth
def assignrul(df):
    maxi = df['cycle'].max()
    df['rul'] = maxi - df['cycle']
    return df
    

train_new = train.groupby('unit').apply(assignrul)

train_new.columns

Index(['unit', 'cycle', 'os1', 'os2', 'os3', 'sm1', 'sm2', 'sm3', 'sm4', 'sm5',
       'sm6', 'sm7', 'sm8', 'sm9', 'sm10', 'sm11', 'sm12', 'sm13', 'sm14',
       'sm15', 'sm16', 'sm17', 'sm18', 'sm19', 'sm20', 'sm21', 'rul'],
      dtype='object')

データフレームには 'RUL' 列があります。この値を予測することがこのワークショップの目的です。

In [4]:
train_new.head(5)

,unit,cycle,os1,os2,os3,sm1,sm2,sm3,sm4,sm5,...,sm13,sm14,sm15,sm16,sm17,sm18,sm19,sm20,sm21,rul
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


最初に注意してほしいのは、0 RUL に近づくにつれてセンサーの測定値が変化しているように見えることです。これは、ビジネス価値にとって十分に役立つモデルを作成できそうだというを意味します。

これで、 Automated ML を使用してこのデータでモデルを訓練する準備が整いました。

## 3. Automated ML

ここでは、Azure Automated ML パッケージを利用してセンサーのスケーリング、センサーの選択を自動化し、さまざまな種類の ML モデルを自動的に訓練して評価します。

Automated ML 用の Azure ML ライブラリをインポートします。

In [5]:
import logging
import os
import random
import time

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from azureml.widgets import RunDetails
from azureml.core.model import Model

Automated ML を実行するには、Machine Learning Workspace の資格情報を入力してください。マイクロソフトの MFA (Multi-Factor Authentication) を実行する必要があります。下のセルを実行して、表示される認証手順に従ってください。

**訳注**  
下のセルを実行する前に、Azure ポータルで **Azure Machine Learning ワークスペース** を作成してください。手順は [ここ](https://docs.microsoft.com/ja-jp/azure/machine-learning/service/setup-create-workspace) を参照してください。

また、認証に失敗したという警告 ("Warning: Falling back to use azure cli login credentials.") が表示された場合は、それに続く表示に従ってデバイスログインしてください。下のセルを実行する時点で認証されていないので、警告表示は正しい動作です。

In [ ]:
#subscription_id = "<Your SubscriptionId>" #所有者または共同管理者である必要があります
#resource_group = "<Resource group - new or existing>" #所有者または共同管理者である必要があります
#workspace_name = "<workspace to be created>" #ワークスペース名
#workspace_region = "<azureregion>" #リージョン

subscription_id = "225795c5-6697-4433-bfa7-20c891a0db76" #所有者または共同管理者である必要があります
resource_group = "ganseo201903" #所有者または共同管理者である必要があります
workspace_name = "ganseo201903" #ワークスペース名
workspace_region = "eastus2" #リージョン

# 以下が設定値の例です
#subscription_id = "381b38e9-9840-4719-a5a0-61d9585e1e91" #所有者または共同管理者である必要があります
#resource_group = "automl_nasa_newrg" #所有者または共同管理者である必要があります
#workspace_name = "automatedml_nasa_aznb" #ワークスペース名
#workspace_region = "eastus2" #リージョン

Azure ML ワークスペースを作成します。

In [ ]:
# Workspace クラスをインポートして、Azure ML SDK バージョンを確認します
from azureml.core import Workspace

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,                      
                      exist_ok=True)
ws.get_details()

クラスターで利用するために設定を保存します。

In [ ]:
from azureml.core import Workspace

ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

# Subscription ID、リソースグループ名、ワークスペース名を aml_config/config.json に保存
ws.write_config()

実験名を定義します。

In [ ]:
# 実験の名前を選択してプロジェクトフォルダを指定します。
experiment_name = 'automl-predictive-rul'
project_folder = './sample_projects/automl-demo-predmain'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data = output, index = ['']).T

トレーニングデータを作成します。

In [ ]:
# トレーニングデータを作成する
X_train = train_new.iloc[:,2:26].values
y_train = train_new.iloc[:,26:27].values.astype(int).flatten()

Automated ML がモデルの学習に使用する配列にデータをフォーマットします。例として、以下に X, Yの 1行分を表示します。

In [ ]:
X_train[0]

In [ ]:
y_train[0]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train,
                                                    y_train,
                                                    test_size=0.3,
                                                    random_state=100)

X_valid, X_test, y_valid, y_test = train_test_split(X_valid,
                                                    y_valid,
                                                    test_size=0.3,
                                                    random_state=100)

X_train = pd.DataFrame(X_train)
X_valid = pd.DataFrame(X_valid)
X_test = pd.DataFrame(X_test)

In [ ]:
X_test[0:1].values.tolist()

テレメトリを有効にします。

In [ ]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics = True)

これで Automated ML を設定する準備が整いました。予測したいもの、使用したい精度メトリック、試したいモデルの数、その他のパラメータに関する情報を提供します。Automated ML は自動的にデータをスケールします。

## Automated ML を設定する

これらのパラメータを使用することができます。  

|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric**|This is the metric that you want to optimize. Classification supports the following primary metrics: <br><i>accuracy</i><br><i>AUC_weighted</i><br><i>average_precision_score_weighted</i><br><i>norm_macro_recall</i><br><i>precision_score_weighted</i>|
|**primary_metric**|This is the metric that you want to optimize. Regression supports the following primary metrics: <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>|
|**iteration_timeout_minutes**|Time limit in minutes for each iteration.|
|**iterations**|Number of iterations. In each iteration AutoML trains a specific pipeline with the data.|
|**n_cross_validations**|Number of cross validation splits.|
|**X**|(sparse) array-like, shape = [n_samples, n_features]|
|**y**|(sparse) array-like, shape = [n_samples, ], [n_samples, n_classes]<br>Multi-class targets. An indicator matrix turns on multilabel classification. This should be an array of integers.|
|**path**|Relative path to the project folder. AutoML stores configuration files for the experiment under this folder. You can specify a new empty folder.|
|**preprocess**|set this to True to enable pre-processing of data eg. string to numeric using one-hot encoding|
|**exit_score**|Target score for experiment. It is associated with the metric. eg. exit_score=0.995 will exit experiment after that|

In [ ]:
## ローカルコンピュート
Automl_config = AutoMLConfig(task = 'regression',
                             primary_metric = 'r2_score',
                             iteration_timeout_minutes = 5,
                             iterations = 3,
                             max_cores_per_iteration = 1,
                             preprocess = False,
                             experiment_exit_score = 0.985,
                             #blacklist_models = ['kNN','LinearSVM','RandomForestRegressor'],
                             whitelist_models = ['ExtremeRandomTrees','ElasticNet','LightGBM'],
                             X = X_train,
                             y = y_train,
                             X_valid = X_valid,
                             y_valid = y_valid,
                             #n_cross_validations = 3,
                             debug_log = 'automl_errors.log',
                             verbosity=logging.ERROR,
                             path=project_folder)

以上で、Automated ML を実行する準備が整いました。この手順には数分かかることがありますが、Automated ML では、モデルが上記で指定したメトリックによってトレーニングおよび評価される間でも最新情報が提供されます。Automated ML は、どのスケーリング方法が使用されたのかも教えてくれます。各 ML モデルトレーニングからの情報は、ML ワークスペースの実験セクションに格納され、Azure Portal を通して確認できます。

In [ ]:
# Automated ML を使って、複数のモデルを学習する
experiment=Experiment(ws, experiment_name)
local_run = experiment.submit(Automl_config, show_output=True)

Widget UX

**訳注** ノートブックを Jupyter Lab で実行している場合には、次のセルは失敗するかもしれません。

In [ ]:
RunDetails(local_run).show()

以上で、最良のモデルを取得できました。

In [ ]:
# 最高の精度を持つ実行を探す
best_run, fitted_model = local_run.get_output()
print(best_run)

## 3. モデルの配置

In [ ]:
# 最良のモデルを登録する
description = 'AutoML-RUL-Regression-20190213'
tags = None
local_run.register_model(description=description, tags=tags)
local_run.model_id # Use this id to deploy the model as a web service in Azure

ML ワークスペースにモデルを登録すると、Azure Portal に表示されます。

今度はモデルを REST API としてデプロイして、1行または複数行の "X" データを指定して、予測された 'RUL' 値を返すことができます。これを実現するために、AML ワークスペースにコンテナーイメージを作成し、そのイメージを Azure の ACI サービスの Container インスタンスとして展開します。その後、データを送信して予測された「RUL」値を受け取る IPアドレスを取得します。

必要なものは3つあります: 
1. A score.py: init および run 関数を含み、モデルをロードしてモデルを作成します
2. A myenv.yml: モデルを実行する python 環境に関する情報を含みます
3. コンテナーイメージおよびサービスのための設定ファイル: Azure ML サービスによって提供されます

以下のセルでこれらを設定します。**上記のセルで提供されている登録モデル名** を使用する必要があります。

**訳注** 登録モデル名はすぐ上のセルの実行結果に表示されています。

In [ ]:
%%writefile score.py
# スコアリングするスクリプト
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

import azureml.train.automl

def init():
    global model
    # モデル名を使ったモデルファイルのパスを取得する
    model_path = Model.get_model_path('<<modelid>>')  # <<modelid>> の部分は実際の登録モデル名で置換
    print(model_path)
    model = joblib.load(model_path)
    

def run(raw_data):
    # データを準備委する
    data = (np.array(json.loads(raw_data)['data'])).reshape(1,-1)
    # 予測
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn','lightgbm'], pip_packages=['azureml-sdk[automl]'])

conda_env_file_name = 'myenv.yml'
myenv.save_to_file('.', conda_env_file_name)

In [ ]:
with open("myenv.yml","r") as f:
    print(f.read())

**もし、上のファイルの pip セクションで "- azureml-train-automl" が表示されない場合は、手動で追加する必要があります。**

**訳注**
azureml-train-automl は下のセルでインストールできます。インストール済みの状態で下のセルを実行しても、問題ありません。

In [ ]:
# 追加インストール
!pip install azureml-train-automl

ここで、Webサービスを設定します。

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, 
                                               memory_gb=2, 
                                               tags={"data": "RUL",  "method" : "sklearn"}, 
                                               description='Predict RUL with Azure AutoML')

最後に、コンテナイメージを設定してサービスをデプロイします。ファイル名が一致し、ワークスペースが変数 ws 設定されており、モデル名が正しいことを確認してください。コンテナーイメージが作成され、Webサービスとして配置されます。

このプロセスには最大10分ほどかかることがあるので、しばらくお待ちください。定期的にプログレスバーを確認してみてください。

**訳注**  
<<modelid>> の部分は、実際の登録モデル名で置換してください。

In [ ]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# コンテナーイメージを設定する
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

service = Webservice.deploy_from_model(workspace=ws,
                                       name='automl-rul-regress',
                                       deployment_config=aciconfig,
                                       models=['<<modelid>>:1'],  # <<modelid>> の部分は実際の登録モデル名で置換
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

確認のために、スコアリング関数の URI を取得できます。

In [ ]:
print(service.scoring_uri)

サービスが機能しているかどうかを確認しましょう。ここで、X_train から単一行のデータを送信して、それが妥当な予測を返すかどうかを確認します。

In [ ]:
import requests
import json

# スコアリング用に、テストセットからランダムに行を選択
#random_index = np.random.randint(0, len(X_train)-1)
input_data = "{\"data\": " + str(X_test[1:2].values.tolist()) + "}" #str(list(X_train[0].reshape(1,-1)[0])) + "}"

headers = {'Content-Type':'application/json'}

# AKS デプロイには、ヘッダーに Service Key が必要
# api_key = service.get_key()
# headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key)} 

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print("input data:", input_data)
print("label:", y_test[1:2])
print("prediction:", resp.text)

多数のフライト、つまりサイクルを経て、エンジンの状態が変わることが分かります。故障に近づくにつれて、予測通り "RUL" がゼロに近づきます。これは、予測モデルがエンジンの今後の故障の予測にどのように役立つかを示す良い例です。

"RUL" の値が大きい場合は、このモデルがうまく機能しません。故障がまだ先なので、これは許容できる結果です。

これ以降 Azure の課金が発生しないように、不要になったサービスを削除します。

In [ ]:
service.delete()